# Build a RA2CE model for FloodAdapt anywhere on the globe

In this notebook we demonstrate how to setup a RA2CE model for the FloodAdapt backend at any given location. As an example we will use the Humber delta in the UK.

## **Step 0:** Import dependencies

In [1]:
from pathlib import Path
import geopandas as gpd